In [1]:
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
import cv2
import os
import numpy as np
# flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2, 'sunflower':3, 'tulip':4} )

Using TensorFlow backend.


In [2]:
category_image = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
resize_width = 200
resize_height = 200
batch_size = 10 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = len(category_image) # 類別的數量
epochs = 500 # 訓練整個資料集共 30個循環
train_val_ratio = 0.8 # 訓練驗證資料比

In [3]:
# 取得圖片總數量
root_dir_path = os.getcwd() + './Day_101_105_ml100marathon_final_exam/train/'
image_numbers = 0
for root, dirs, files in os.walk(root_dir_path):
    for f in files:
        image_numbers += 1

In [4]:
# 讀取訓練圖片並調整大小
x = np.empty(shape=[image_numbers, resize_width, resize_height, 3], dtype='float32')
y = np.empty(shape=[image_numbers, 1], dtype='float32')

current_number = 0

for category_index, category_name in enumerate(category_image):
    image_dir_path = os.getcwd() + './Day_101_105_ml100marathon_final_exam/train/' + category_name+ '/'
    image_filename = os.listdir(image_dir_path)

    for i in range(len(image_filename)):
        img = load_img(image_dir_path + image_filename[i])
        arr_img = img_to_array(img)
        arr_img_resized = cv2.resize(arr_img, (resize_width, resize_height))
        x[current_number] = arr_img_resized
        y[current_number] = category_index
        current_number += 1

In [5]:
# 建立訓練和驗證資料
import random
shuffle_index = list(range(image_numbers))
random.shuffle(shuffle_index)
x_temp = x[shuffle_index]
y_temp = y[shuffle_index]

x_train = x_temp[0:int(len(x)*train_val_ratio)]
x_val = x_temp[int(len(x)*train_val_ratio):]
y_train = y_temp[0:int(len(x)*train_val_ratio)]
y_val = y_temp[int(len(x)*train_val_ratio):]

x_train = x_train / 255.
x_val = x_val / 255.
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)

In [6]:
# 建立 ResNet 模型
model = resnet(input_shape=(resize_width, resize_height, 3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 200, 200, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 200, 200, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 200, 200, 16) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
# 載入 Callbacks
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor="val_loss", 
                          patience=5, 
                          verbose=1
                          )

In [8]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=5, 
                              verbose=1)

In [9]:
from keras.preprocessing.image import ImageDataGenerator
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val))
                    #callbacks=[earlystop, reduce_lr])

#history = model.fit(x_train, y_train,
#                    batch_size=batch_size,
#                    epochs=epochs,
#                    verbose=1,
#                    validation_data=(x_val, y_val))

score = model.evaluate(x_val, y_val, verbose=0)
print('Valuation loss:', score[0])
print('Valuation accuracy:', score[1])

Epoch 1/500
225/225 [==============================] - 42s 188ms/step - loss: 0.9966 - acc: 0.6844 - val_loss: 1.2924 - val_acc: 0.6283
Epoch 2/500
225/225 [==============================] - 37s 164ms/step - loss: 0.9613 - acc: 0.7017 - val_loss: 0.9462 - val_acc: 0.6885
Epoch 3/500
225/225 [==============================] - 37s 164ms/step - loss: 0.9519 - acc: 0.7071 - val_loss: 0.9784 - val_acc: 0.6973
Epoch 4/500
225/225 [==============================] - 37s 165ms/step - loss: 0.9781 - acc: 0.6933 - val_loss: 0.9498 - val_acc: 0.7310
Epoch 5/500
225/225 [==============================] - 37s 165ms/step - loss: 0.9237 - acc: 0.7071 - val_loss: 1.1058 - val_acc: 0.6531
Epoch 6/500
225/225 [==============================] - 37s 165ms/step - loss: 0.9138 - acc: 0.7161 - val_loss: 0.9410 - val_acc: 0.7150
Epoch 7/500
225/225 [==============================] - 37s 165ms/step - loss: 0.9197 - acc: 0.7114 - val_loss: 1.1133 - val_acc: 0.6478
Epoch 8/500
225/225 [===========================

Epoch 61/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6577 - acc: 0.8114 - val_loss: 0.8959 - val_acc: 0.7540
Epoch 62/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6364 - acc: 0.8101 - val_loss: 0.8264 - val_acc: 0.7628
Epoch 63/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6820 - acc: 0.7968 - val_loss: 0.7345 - val_acc: 0.7823
Epoch 64/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6438 - acc: 0.8120 - val_loss: 0.9285 - val_acc: 0.7416
Epoch 65/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6139 - acc: 0.8132 - val_loss: 0.8040 - val_acc: 0.7947
Epoch 66/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6819 - acc: 0.7921 - val_loss: 0.8467 - val_acc: 0.7947
Epoch 67/500
225/225 [==============================] - 37s 165ms/step - loss: 0.6429 - acc: 0.8156 - val_loss: 0.9021 - val_acc: 0.7469
Epoch 68/500
225/225 [===================

225/225 [==============================] - 37s 165ms/step - loss: 0.4918 - acc: 0.8577 - val_loss: 0.7952 - val_acc: 0.8124
Epoch 180/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4869 - acc: 0.8764 - val_loss: 0.7564 - val_acc: 0.7894
Epoch 181/500
225/225 [==============================] - 37s 165ms/step - loss: 0.5026 - acc: 0.8611 - val_loss: 1.0637 - val_acc: 0.7345
Epoch 182/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4973 - acc: 0.8617 - val_loss: 0.7906 - val_acc: 0.7858
Epoch 183/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4824 - acc: 0.8711 - val_loss: 1.0825 - val_acc: 0.7876
Epoch 184/500
225/225 [==============================] - 37s 165ms/step - loss: 0.5125 - acc: 0.8621 - val_loss: 1.0953 - val_acc: 0.7664
Epoch 185/500
225/225 [==============================] - 37s 165ms/step - loss: 0.5003 - acc: 0.8581 - val_loss: 0.8253 - val_acc: 0.7912
Epoch 186/500
225/225 [=========================

225/225 [==============================] - 37s 165ms/step - loss: 0.4419 - acc: 0.8924 - val_loss: 0.7931 - val_acc: 0.8124
Epoch 298/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4115 - acc: 0.9017 - val_loss: 0.7908 - val_acc: 0.8142
Epoch 299/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4513 - acc: 0.8837 - val_loss: 0.9531 - val_acc: 0.8071
Epoch 300/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4051 - acc: 0.9017 - val_loss: 0.8102 - val_acc: 0.8195
Epoch 301/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4105 - acc: 0.8938 - val_loss: 0.9068 - val_acc: 0.8071
Epoch 302/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4315 - acc: 0.8932 - val_loss: 0.9406 - val_acc: 0.7929
Epoch 303/500
225/225 [==============================] - 37s 165ms/step - loss: 0.4117 - acc: 0.8954 - val_loss: 1.0799 - val_acc: 0.8000
Epoch 304/500
225/225 [=========================

225/225 [==============================] - 37s 165ms/step - loss: 0.3754 - acc: 0.9073 - val_loss: 1.0091 - val_acc: 0.8142
Epoch 416/500
225/225 [==============================] - 37s 165ms/step - loss: 0.3793 - acc: 0.9139 - val_loss: 0.8703 - val_acc: 0.8212
Epoch 417/500
225/225 [==============================] - 37s 165ms/step - loss: 0.3873 - acc: 0.9120 - val_loss: 0.7643 - val_acc: 0.8265
Epoch 418/500
225/225 [==============================] - 37s 165ms/step - loss: 0.3601 - acc: 0.9227 - val_loss: 0.8774 - val_acc: 0.8106
Epoch 419/500
225/225 [==============================] - 37s 165ms/step - loss: 0.3508 - acc: 0.9229 - val_loss: 1.0376 - val_acc: 0.7894
Epoch 420/500
225/225 [==============================] - 37s 165ms/step - loss: 0.3610 - acc: 0.9199 - val_loss: 0.8904 - val_acc: 0.8372
Epoch 421/500
225/225 [==============================] - 37s 165ms/step - loss: 0.3727 - acc: 0.9160 - val_loss: 0.9002 - val_acc: 0.8035
Epoch 422/500
225/225 [=========================

In [12]:
# 取得測試圖片總數量
root_dir_path_test = os.getcwd() + './Day_101_105_ml100marathon_final_exam/test/'
image_numbers_test = 0
for root, dirs, files in os.walk(root_dir_path_test):
    for f in files:
        image_numbers_test += 1

In [20]:
# 讀取測試圖片並調整大小
x_test = np.empty(shape=[image_numbers_test, resize_width, resize_height, 3], dtype='float32')
filename_test = []
current_number_test = 0
image_filename_test = os.listdir(root_dir_path_test)

for i in range(len(image_filename_test)):
    img = load_img(root_dir_path_test + image_filename_test[i])
    arr_img = img_to_array(img)
    arr_img_resized = cv2.resize(arr_img, (resize_width, resize_height))
    x_test[current_number_test] = arr_img_resized
    current_number_test += 1
    filename_test.append(os.path.splitext(image_filename_test[i])[0])
    
x_test = x_test / 255.

In [13]:
# 預測結果
y_pred = model.predict(x_test)

In [15]:
y_classes = [np.argmax(y, axis=None, out=None) for y in y_pred] 

In [27]:
import pandas as pd
dict_result = {"id" : filename_test, "flower_class" : y_classes}
result = pd.DataFrame(dict_result)

In [30]:
result.to_csv('ML100marathon_final_exam_result.csv', index=False)